In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.externals import joblib
from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import accuracy_score,recall_score,f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

## 上机任务 对HR表进行knn算法学习和预测，要求对数据进行不同方式的变换，对比不同变换下预测结果的好坏，在knn学习时，也要求对neighbors设置不同的值，对比预测结果的好坏

In [8]:
d=dict([("low",0),("medium",1),("high",2)])
def map_salary(s):
    return d.get(s,0)
#sl:satisfaction_level----False:MinMaxScaler; True: StandardScaler
#le:last_evaluation----False:MinMaxScaler; True: StandardScaler
def hr_preprocessing(sl=False,le=False,npr=False,amh=False,tsc=False,wa=False,dp=False,slr=False,lower_d=False,ld_n=1):
    df=pd.read_csv("D:\python3.7\HEHE\HR.csv",engine='python')
    #数据清洗
    df=df.dropna(subset=["satisfaction_level","last_evaluation"])
    df=df[df["satisfaction_level"]<=1][df["salary"]!="nme"]
    
    #获得标注
    label=df["left"]
    df=df.drop("left",axis=1)
    
    #特征变换
    scaler_lst=[sl,le,npr,amh,tsc,wa]
    column_lst=["satisfaction_level","last_evaluation","number_project","average_monthly_hours","time_spend_company","Work_accident"]
    
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            df[column_lst[i]]=MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
        else:
            df[column_lst[i]]=StandardScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
            
    scaler_lst=[slr,dp]
    column_lst=["salary","department"]
    
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            if column_lst[i]=="salary":
                df[column_lst[i]]=[map_salary(s) for s in df["salary"].values]
            else:
                df[column_lst[i]]=LabelEncoder().fit_transform(df[column_lst[i]])
                df[column_lst[i]]=MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
        else:
            df=pd.get_dummies(df,columns=[column_lst[i]])
            
    #降维处理
    if lower_d:
        return PCA(n_components=ld_n).fit_transform(df.values),label
    
    return df,label

def Estimator(features,label,neighbors):
    X_train,X_test,Y_train,Y_test=train_test_split(features,label,test_size=0.2)
    knn_clf=KNeighborsClassifier(neighbors)
    knn_clf.fit(X_train,Y_train)
    Y_pred=knn_clf.predict(X_test)
    print("正确率ACC:",accuracy_score(Y_test,Y_pred))
    print("召回率REC:",recall_score(Y_test,Y_pred,average='micro'))
    print("查准率F-score:",f1_score(Y_test,Y_pred,average='micro'))

In [9]:
#features,label=hr_preprocessing()        #第一种变换
#features,label=hr_preprocessing(sl=True,le=True,npr=True,amh=True,tsc=True,wa=True,dp=True,slr=False,lower_d=False,ld_n=1) #第二种变换
features,label=hr_preprocessing(dp=True) #第三种变换
#Estimator(features,label,neighbors=5)  #第一种neighbors值的设置
Estimator(features,label,neighbors=3)  #第二种neighbors值的设置

D:\Users\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


正确率ACC: 0.957
召回率REC: 0.957
查准率F-score: 0.957
